In [27]:
import spacy 
import gzip
import re 
import pandas as pd
from spacy.lang.en import English
from collections import Counter
from itertools import chain
import ahocorasick
import ast
from analyze_m_data import get_occupations_list_en, get_occupations_list
from spacy.lang.de import German
from spacy.lang.it import Italian
from spacy.lang.es import Spanish 
import matplotlib.pyplot as plt
import itertools
from collections import Counter
from operator import itemgetter

In [14]:
filename = "/mount/studenten/projects/caulfiea/cross-verified-database.csv.gz"
csvFilename = gzip.open(filename, 'rb')
df = pd.read_csv(csvFilename, encoding='latin-1')
print(df.head(10))
print(df.columns)

  wikidata_code   birth   death  updated_death_date approx_birth approx_death  \
0      Q1000002  1932.0  1990.0                 NaN          NaN          NaN   
1      Q1000005  1860.0  1927.0                 NaN          NaN          NaN   
2      Q1000006  1971.0     NaN                 NaN          NaN          NaN   
3      Q1000015  1983.0     NaN                 NaN          NaN          NaN   
4      Q1000023  1912.0  1977.0                 NaN          NaN          NaN   
5      Q1000026  1928.0  2016.0                 NaN          NaN          NaN   
6      Q1000034  1818.0  1894.0                 NaN          NaN          NaN   
7      Q1000044  1907.0  1980.0                 NaN          NaN          NaN   
8      Q1000045  1915.0  1987.0                 NaN          NaN        circa   
9      Q1000048  1891.0  1962.0                 NaN          NaN          NaN   

   birth_min  birth_max  death_min  death_max  ... area2_of_rattachment  \
0     1932.0     1932.0     1990.

In [4]:
print(df.columns)

Index(['wikidata_code', 'birth', 'death', 'updated_death_date', 'approx_birth',
       'approx_death', 'birth_min', 'birth_max', 'death_min', 'death_max',
       'gender', 'level1_main_occ', 'name', 'un_subregion', 'birth_estimation',
       'death_estimation', 'bigperiod_birth_graph_b',
       'bigperiod_death_graph_b', 'curid', 'level2_main_occ', 'freq_main_occ',
       'freq_second_occ', 'level2_second_occ', 'level3_main_occ',
       'bigperiod_birth', 'bigperiod_death', 'wiki_readers_2015_2018',
       'non_missing_score', 'total_count_words_b', 'number_wiki_editions',
       'total_noccur_links_b', 'sum_visib_ln_5criteria',
       'ranking_visib_5criteria', 'all_geography_groups',
       'string_citizenship_raw_d', 'citizenship_1_b', 'citizenship_2_b',
       'list_areas_of_rattach', 'area1_of_rattachment', 'area2_of_rattachment',
       'list_wikipedia_editions', 'un_region', 'group_wikipedia_editions',
       'bplo1', 'dplo1', 'bpla1', 'dpla1', 'pantheon_1', 'level3_all_occ'],
 

In [99]:
english_female_df = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/en_female_occ_all_sentences_full_data.csv', encoding="utf-8")
en_female_filename = "/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/datasets/final_sets/en_female_dataset.csv"
en_female_df = pd.read_csv(en_female_filename, names= ['wikidata_code', 'title', 'intro', 'gender', 'occupations'],header=0, encoding='utf-8')


In [100]:
en_female_codes = english_female_df['wikidata_code'].to_list()
df2 = df.loc[df['wikidata_code'].isin(en_female_codes)]
print(df2.shape[0])

102444


In [101]:
notability_scores = df2[['wikidata_code','number_wiki_editions','sum_visib_ln_5criteria']]
result = pd.merge(english_female_df, notability_scores, on='wikidata_code')
print(result.head())

   Unnamed: 0 wikidata_code                         title  \
0           0       Q101138           elisabeth bauriedel   
1           1     Q10305226  janet auchincloss rutherfurd   
2           2     Q10320901                 luiza mariani   
3           3     Q10325735         maria baptista soares   
4           4     Q10346103                 patricia seed   

                                               intro  gender  \
0  elisabeth bauriedel (born 20 june 1939) was a ...  female   
1  janet jennings auchincloss rutherfurd (june 13...  female   
2  luiza mariani is a brazilian actress and produ...  female   
3  maria baptista soares (born november 15, 1928)...  female   
4  patricia seed is an american historian and pro...  female   

                           occupations  \
0                       ['politician']   
1                        ['socialite']   
2                            ['actor']   
3                           ['writer']   
4  ['historian', 'university teacher']

In [60]:
print(result.columns)

Index(['Unnamed: 0', 'wikidata_code', 'title', 'intro', 'gender',
       'occupations', 'first_sentence', 'named_entities',
       'overlapping_occupations', 'overlapping_occupations_sentence1',
       'number_wiki_editions', 'sum_visib_ln_5criteria'],
      dtype='object')


In [29]:
def get_split_occupations_en(filename):
    new_df = pd.read_csv(filename, names= ['occupation'],header=0, encoding='utf-8')
    new_df["occupation"] = new_df['occupation'].astype(str)
    # lowercase everything 
    new_df['occupation'] = new_df['occupation'].str.casefold()
    # get rid of extra quotation marks
    new_df['occupation'] = new_df['occupation'].apply(lambda x: x.replace('"', ''))
    # split on commas 
    new_df['occupation'] = new_df['occupation'].apply(lambda x: x.split(','))
    # only take first item in the list split on commas
    new_df['occupation'] = new_df['occupation'].str[0]
    # print(new_df['occupation'].head(20))
    unique_occupations = set(new_df['occupation'].to_list())
    
    # separate strings with and without whitespace
    no_whitespace = [s for s in unique_occupations if " " not in s]
    with_whitespace = [s for s in unique_occupations if " " in s]
    print("All:", len(unique_occupations))
    print("No Whitespace:", len(no_whitespace))
    print("With Whitespace:", len(with_whitespace))
    # print(len(unique_occupations))
    # for i, val in enumerate(itertools.islice(unique_occupations, 10)):
    #     print(val)
    return no_whitespace,with_whitespace

In [102]:
def new_english_analysis(df):
    #Want to: 
    # 1. Split up english occupation titles list into two lists: one token occupations and multi token
    # 2. For one token occupations, want to find matches (in first sentence) with hard word boundaries (like in german code)
    # 3. For multi token occupations, want to just look up matches 
    # 4. Want to remove named entities from matched tokens in that line 
    # 5. Want to get final matched titles list 
    en_1token_occ_titles, en_multitoken_occ_titles = get_split_occupations_en('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/occ_title-lists/eng_occ_titles.csv')
    print(en_1token_occ_titles)
    df['intro'] = df['intro'].astype(str)
    df['first_sentence'] = df['first_sentence'].astype(str)
    # df['named_entities'] = df['named_entities'].astype(str)
    df['named_entities'] = df['named_entities'].apply(ast.literal_eval)
    # df['named_entities'] = df['named_entities'].str.casefold
    df['named_entities'] = df['named_entities'].apply(lambda x: [item.lower() for item in x])
    print(df['named_entities'].head(10))
    # add wikidata occupations to one token titles
    # df['occupations'] = df['occupations'].apply(ast.literal_eval)
    # wiki_occupations = set(chain.from_iterable(df['occupations']))
    # en_1token_occ_titles = en_1token_occ_titles.extend(wiki_occupations)

    # get matching tokens for one word tokens 
    automaton = ahocorasick.Automaton()
    for id, term in enumerate(en_1token_occ_titles):
        automaton.add_word(term, (term))
    automaton.remove_word('-')
    automaton.remove_word('former')
    automaton.remove_word('first')
    automaton.remove_word('second')
    automaton.make_automaton()
    def find_terms_en(text):
        words = list(re.findall(r'\b\w+\b', text.lower()))  # Tokenize text into words
        matches = list(term for _, term in automaton.iter(text) if term.lower() in words)
        return matches
    
    # get matches for multi word tokens (simpler lookup)
    automaton_multi = ahocorasick.Automaton()
    for id, term in enumerate(en_multitoken_occ_titles):
        automaton_multi.add_word(term, (term))
    automaton_multi.remove_word('-')
    automaton_multi.make_automaton()
    def find_terms_en_multi(text):
        return list(term for _, term in automaton_multi.iter(text))
    all_occ_titles = []
    all_occ_titles.extend(en_multitoken_occ_titles)
    all_occ_titles.extend(en_1token_occ_titles)
    print(all_occ_titles)
    automaton_gendered = ahocorasick.Automaton()
    for id, term in enumerate(all_occ_titles):
        term_female= ''.join(('female',' ',term))
        term_male = ''.join(('male',' ',term))
        automaton_gendered.add_word(term_female, (term_female))
        automaton_gendered.add_word(term_male, (term_male))
    automaton_multi.remove_word('-')
    automaton_gendered.make_automaton()
    def find_terms_en_gendered(text):
        return list(term for _, term in automaton_gendered.iter(text))
    
    def find_all_terms(text): 
        single_terms = find_terms_en(text)
        multi_terms = find_terms_en_multi(text)
        single_terms.extend(multi_terms)
        gendered_terms = find_terms_en_gendered(text)
        single_terms.extend(gendered_terms)
        return single_terms
    
    def filter_NER(row):
        col1_counts = Counter(row['overlapping_occupations_sentence1'])  # Count occurrences in col1
        col2_counts = Counter(row['named_entities'])  # Count occurrences in col2

        # Remove only up to the number of times they appear in col2
        for word, count in col2_counts.items():
            if word in col1_counts:
                col1_counts[word] -= count  # Reduce count but not below zero
                if col1_counts[word] <= 0:
                    del col1_counts[word]  # Remove word if count reaches zero

        # Reconstruct the filtered list based on updated counts
        filtered_list = []
        for word, count in col1_counts.items():
            filtered_list.extend([word] * count)  # Add back only remaining occurrences

        return filtered_list

    df['overlapping_occupations'] = df['intro'].apply(find_all_terms)
    df['overlapping_occupations_sentence1'] = df['first_sentence'].apply(find_all_terms)
    # remove named entities from matched occupations
    df['overlapping_occupations_sentence1'] = df.apply(filter_NER, axis=1)
    print(df['overlapping_occupations_sentence1'].head(10))
    

In [103]:
new_english_analysis(result)
overlapping_occ_counts_s1_female = Counter(chain.from_iterable(result['overlapping_occupations_sentence1']))
print(overlapping_occ_counts_s1_female)

All: 9284
No Whitespace: 1046
With Whitespace: 8238
['interpreter', 'stonecutter', 'policeman', 'fisher', 'sider', 'stripper-cutter', 'machinist', 'winder', 'boiler', 'hooker-laster', 'salesperson-demonstrator', 'gamekeeper', 'matcher', 'packer-fuser', 'biologist', 'acupressurist', 'forester', 'snailer', 'businessman', 'wife', 'glass-worker', 'raker', 'linker', 'sterilizer', 'otolaryngologist', 'racker', 'crusher', 'inspector-repairer', 'housekeeper', 'planner', 'hatter', 'firefighter', 'roller-stitcher', 'wedger', 'geophysicist', 'frit-mixer-and-burner', 'anchorwoman', 'lather', 'cooker', 'embroiderer', 'drier', 'fireman', 'sleever', 'garnetter', 'teller', 'knitter', 'geographer', 'picker', 'rafter', 'header', 'grader', 'proofreader', 'turner', 'waitress', 'timekeeper', 'appliquer', 'microscopist', 'spotter', 'starcher', 'barmaid', 'framer', 'telecommunicator', 'pegger', 'bookmaker', 'cutter-and-paster', 'barman', 'host/hostess', 'barker', 'caponizer', 'businesswoman', 'programmer', '

In [38]:
from collections import defaultdict
# Initialize counters
tag_counts = defaultdict(int)
tag_score_totals = defaultdict(float)

# Iterate over rows
for tags, score in zip(result['overlapping_occupations_sentence1'], result['number_wiki_editions']):
    for tag in tags:
        tag_counts[tag] += 1
        tag_score_totals[tag] += score

# Now compute averages
tag_avg_scores = {
    tag: tag_score_totals[tag] / tag_counts[tag]
    for tag in tag_counts
}

# Combine into a DataFrame if you want
result_df = pd.DataFrame({
    'count': tag_counts,
    'average_score': tag_avg_scores
})

print(result_df)

                    count  average_score
actress              7358       1.037102
comedian              521       1.046065
hostess                71       1.000000
host                  779       1.008986
flight attendant       15       1.000000
actor                 371       1.002695
businesswoman        1105       1.003620
chairman              152       1.006579
headmistress           40       1.025000
businessperson         85       1.011765
chairwoman             35       1.000000
comedienne             17       1.000000
policewoman             9       1.111111
news anchor           200       1.005000
weather forecaster     12       1.000000
clergyman               2       1.000000
businessman            25       1.000000
meteorologist          45       1.000000
stuntwoman             14       1.071429
seamstress             13       1.000000
police officer         76       1.000000
firefighter             6       1.000000
headteacher             6       1.000000
policeman       

In [ ]:
from collections import defaultdict
# Initialize counters
tag_counts = defaultdict(int)
tag_score_totals = defaultdict(float)

# Iterate over rows
for tags, score in zip(result['overlapping_occupations_sentence1'], result['sum_visib_ln_5criteria']):
    for tag in tags:
        tag_counts[tag] += 1
        tag_score_totals[tag] += score

# Now compute averages
tag_avg_scores = {
    tag: tag_score_totals[tag] / tag_counts[tag]
    for tag in tag_counts
}

# Combine into a DataFrame if you want
result_df = pd.DataFrame({
    'count': tag_counts,
    'average_score': tag_avg_scores
})

print(result_df)

                    count  average_score
actress              7358      18.528778
comedian              521      19.040828
hostess                71      18.641826
host                  779      18.534057
flight attendant       15      17.306245
actor                 371      17.516233
businesswoman        1105      17.415664
chairman              152      16.822033
headmistress           40      15.907072
businessperson         85      17.262597
chairwoman             35      16.936125
comedienne             17      18.920599
policewoman             9      15.447511
news anchor           200      18.651814
weather forecaster     12      18.910068
clergyman               2      15.891572
businessman            25      17.515880
meteorologist          45      18.691895
stuntwoman             14      18.144715
seamstress             13      16.599542
police officer         76      16.765910
firefighter             6      15.699137
headteacher             6      15.759665
policeman       

In [53]:
english_male_df = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/en_male_occ_all_sentences_full_data.csv', encoding="utf-8")
en_male_codes = english_male_df['wikidata_code'].to_list()
df3 = df.loc[df['wikidata_code'].isin(en_male_codes)]
print(df3.shape[0])
notability_scores = df3[['wikidata_code','number_wiki_editions','sum_visib_ln_5criteria','wiki_readers_2015_2018','total_noccur_links_b','non_missing_score']]
result_male = pd.merge(english_male_df, notability_scores, on='wikidata_code')
print(result_male.head())

94720
   Unnamed: 0 wikidata_code                                 title  \
0           0     Q25999686                       moisés hipólito   
1           1     Q29021452                        frank m. riley   
2           2     Q46916921                 baddam narsimha reddy   
3           3     Q27924891  thomas perry (australian politician)   
4           4      Q5335262                           ed pearlman   

                                               intro gender  \
0  moisés hipólito león (born 15 march 1992) is a...   male   
1  frank m. riley (september 10, 1875 – may 22, 1...   male   
2  baddam narsimha reddy (21 june 1931 – 6 novemb...   male   
3  thomas oswald perry (17 january 1914 – 8 march...   male   
4  ed pearlman is a co-founder (together with don...   male   

                       occupations  \
0  ['association football player']   
1                    ['architect']   
2                   ['politician']   
3                   ['politician']   
4         

In [54]:
new_english_analysis(result_male)
overlapping_occ_counts_s1_male = Counter(chain.from_iterable(result_male['overlapping_occupations_sentence1']))
print(overlapping_occ_counts_s1_male)

All: 60
No Whitespace: 52
With Whitespace: 8
hello
0    [hipólito, león, 15, march, 1992, mexican, cru...
1    [frank, m., riley, september, 10, ,, 1875, may...
2    [21, june, 1931, november, 2017, indian, the, ...
3    [thomas, oswald, perry, 17, january, 1914, mar...
4    [ed, pearlman, don, francisco, the, national, ...
5    [nicholas, hemming, english, the, early, 1990s...
6    [bryan, ellis, burwell, august, 4, ,, 1955, de...
7    [russ, karpisek, july, 20, ,, 1966, u.s., nebr...
8    [mike, grob, may, 6, ,, 1964, american, the, u...
9    [james, brown, september, 1877, new, zealand, ...
Name: named_entities, dtype: object
0    []
1    []
2    []
3    []
4    []
5    []
6    []
7    []
8    []
9    []
Name: overlapping_occupations_sentence1, dtype: object
Counter({'businessman': 2432, 'actor': 2058, 'chairman': 735, 'clergyman': 406, 'comedian': 346, 'host': 329, 'police officer': 121, 'businessperson': 84, 'headmaster': 53, 'meteorologist': 44, 'news anchor': 44, 'stuntman': 26,

In [ ]:
# Initialize counters
tag_counts = defaultdict(int)
tag_score_totals = defaultdict(float)

# Iterate over rows
for tags, score in zip(result_male['overlapping_occupations_sentence1'], result_male['total_noccur_links_b']):
    for tag in tags:
        tag_counts[tag] += 1
        tag_score_totals[tag] += score

# Now compute averages
tag_avg_scores = {
    tag: tag_score_totals[tag] / tag_counts[tag]
    for tag in tag_counts
}

# Combine into a DataFrame if you want
result_df = pd.DataFrame({
    'count': tag_counts,
    'average_score': tag_avg_scores
})

print(result_df)

                    count  average_score
clergyman             406      16.971334
businessman          2432      16.924932
host                  329      18.512553
actor                2058      18.447877
chairman              735      16.988803
comedian              346      18.799908
police officer        121      16.745028
policeman              14      15.647713
meteorologist          44      17.029830
news anchor            44      18.445023
businessperson         84      15.873361
firefighter            18      17.042866
salesman               20      16.565470
cameraman              13      17.308093
headmaster             53      16.302429
fireman                10      16.740692
stuntman               26      18.044722
weather forecaster      8      18.413049
crew member             3      19.837390
steward                 9      17.114840
mailman                 1      21.128130
weatherman              4      17.712492
landlord                9      18.150796
actress         

In [55]:
import numpy as np
tag_scores = defaultdict(list)

# Collect all scores per tag
for tags, score in zip(result_male['overlapping_occupations_sentence1'], result_male['non_missing_score']):
    for tag in tags:
        tag_scores[tag].append(score)

# Compute statistics
tag_stats = {
    tag: {
        'count': len(scores),
        'average_score': np.mean(scores),
        'median_score': np.median(scores),
        'min_score': np.min(scores),
        'max_score': np.max(scores)
    }
    for tag, scores in tag_scores.items()
}

# Create DataFrame and sort
result_df = pd.DataFrame.from_dict(tag_stats, orient='index')
result_df = result_df.sort_values(by='average_score', ascending=False)

print(result_df)

                    count  average_score  median_score  min_score  max_score
crew member             3       3.000000           3.0          3          3
fireman                10       3.000000           3.0          3          3
headmistress            1       3.000000           3.0          3          3
chairwoman              1       3.000000           3.0          3          3
hostess                 1       3.000000           3.0          3          3
bartender               1       3.000000           3.0          3          3
mail carrier            1       3.000000           3.0          3          3
headteacher             1       3.000000           3.0          3          3
server                  1       3.000000           3.0          3          3
weatherman              4       3.000000           3.0          3          3
mailman                 1       3.000000           3.0          3          3
weather forecaster      8       3.000000           3.0          3          3